In [3]:
import os
from __init__ import *

__ORIG_WD__ = os.getcwd()
os.chdir(f"{__ORIG_WD__}/../data_collectors")

from covid19_genome import Covid19Genome

os.chdir(__ORIG_WD__)


In [4]:
dc = Covid19Genome()

Building Data frame
Done building Data frame
Building remote dicts
Done building remote dicts
Building local dicts
Done building local dicts


In [ ]:
lineages = dc.getLocalLineages()

mappings = []
accessions = for lineage in lineages:
    for accession in dc.getLocalAccessionsPaths(lineage):
        mappings.append((accession, lineage))


In [8]:
import tensorflow as tf

def compute_coverage(n, indexes, read_lengths):
    # Create a range tensor [0, 1, 2, ... n-1]
    range_tensor = tf.range(n, dtype=tf.int32)

    # Expand dims for broadcasting
    expanded_range = tf.expand_dims(range_tensor, 0)
    expanded_indexes = tf.expand_dims(indexes, 1)
    expanded_lengths = tf.expand_dims(read_lengths, 1)

    # Create a binary mask for each read
    # Each row in the mask represents where the read starts and its length
    print((expanded_range >= expanded_indexes).shape)
    mask = tf.logical_and(expanded_range >= expanded_indexes,
                          expanded_range < expanded_indexes + expanded_lengths)

    # Convert the boolean mask to integers and sum across rows
    coverage = tf.reduce_sum(tf.cast(mask, tf.int32), axis=0)
    
    return coverage

# Example
n = 10
indexes = tf.constant([1, 2, 5], dtype=tf.int32)
read_lengths = tf.constant([3, 20, 2], dtype=tf.int32)

coverage_per_base = compute_coverage(n, indexes, read_lengths)
print(coverage_per_base.numpy())  # Expected: [0, 1, 1, 1, 0, 1, 1, 0, 0, 0]


(3, 10)
[0 1 2 2 1 2 2 1 1 1]


In [11]:
tf.tensor_scatter_nd_add(
    tf.zeros([3], tf.constant([0], [0]), tf.constant([1,1]))
)

TypeError: Cannot convert the argument `type_value`: [0] to a TensorFlow DType.

In [1]:
import tensorflow as tf

def propagate_values(vector):
    # Find where the vector is not zero
    non_zeros = tf.cast(tf.not_equal(vector, 0), dtype=tf.int32)

    # Get the cumulative sum of non-zero locations
    cum_sum = tf.cumsum(non_zeros)

    # Create a mask to gather elements
    mask = cum_sum * non_zeros

    # Use tf.gather to replace zeros with latest non-zero value
    propagated_values = tf.gather(tf.boolean_mask(vector, mask), cum_sum - 1)

    return propagated_values

# Example
vec = tf.constant([0, 0, 0, -1, 0, 0, 0, -2, 0, 0, 0], dtype=tf.int32)
result = propagate_values(vec)
print(result.numpy())  # Expected: [0,0,0,3,3,3,3,3,8,8,8]


2023-08-29 14:56:23.750716: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 14:56:23.819231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 14:56:25.214459: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-29 14:56:26.678762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

[ 0  0  0 -1 -1 -1 -1 -2 -2 -2 -2]


2023-08-29 14:56:26.720511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-29 14:56:26.720751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-29 14:56:26.723778: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
a = tf.constant([1,2,3,4,5])
tf.diff(a)

AttributeError: module 'tensorflow' has no attribute 'diff'

In [31]:
def random_partition(balls, bins):
    """
    1 represents a ball
    0 represents a partition
    """
    # randomize the ones indexes
    balls_indexes = tf.random.shuffle(tf.range(balls+bins-1))[:balls]
    balls_and_partitions = tf.concat([
        tf.scatter_nd(
            indices=tf.expand_dims(balls_indexes, 1),
            updates=tf.ones(balls, dtype=tf.dtypes.int32),
            shape=[bins+balls-1]
        ), 
        [0]
    ], axis=-1)
    
    # cumsum of balls and partitions
    cumsum = tf.cumsum(balls_and_partitions)

    summed_balls_per_partition = tf.boolean_mask(cumsum, balls_and_partitions == 0)

    prev_balls_per_partition = tf.concat([[0], summed_balls_per_partition[:-1]], axis = -1)

    return summed_balls_per_partition - prev_balls_per_partition

In [43]:
def _compute_coverage_per_base(genome_length, read_length, coverage):
    num_reads = tf.cast(tf.math.ceil(coverage * genome_length / read_length), tf.int32)
    read_starts_per_base = tf.concat(
        [
            random_partition(
                balls = num_reads,
                bins = genome_length - read_length + 1,
            ),
            tf.zeros(read_length, dtype=tf.int32)
        ],
        axis=-1
    )
    tf.print(read_starts_per_base)
    read_ends_per_base = -1*tf.roll(read_starts_per_base, read_length, axis=0)
    tf.print(read_ends_per_base)
    coverage_per_base = tf.cumsum(read_starts_per_base + read_ends_per_base)
    return coverage_per_base

In [44]:
_compute_coverage_per_base(5, 2, 5)

[6 1 3 3 0]
[0 -6 -1 -3 -3]


<tf.Tensor: shape=(5,), dtype=int32, numpy=array([6, 1, 3, 3, 0], dtype=int32)>